Start docker container with elasticsearch:
_docker pull docker.elastic.co/elasticsearch/elasticsearch:7.12.1
_docker run -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:7.12.1

In [ ]:
from TEI_Handling import TEIFile
import glob
from pathlib import Path
import multiprocessing
from multiprocessing.pool import Pool
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
from haystack.preprocessor import PreProcessor

In [ ]:
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

In [ ]:
def to_dict(paper_path):
    tei = TEIFile(paper_path)
    processor = PreProcessor(
        clean_empty_lines=True,
        clean_whitespace=True,
        clean_header_footer=True,
        split_by="word",
        split_length=250,
        split_respect_sentence_boundary=True,
        split_overlap=20
    )
    return processor.process(tei.to_dict())

In [ ]:
papers = sorted(Path("../data/unpaywall-grobid-sample").glob('*.tei.xml'))
print(f"Processing {papers.__len__()} papers on {multiprocessing.cpu_count()} cores.")
pool = Pool()

In [ ]:
dicts = []
dicts.extend(pool.imap(to_dict, papers, 5))
pool.close()

In [ ]:
documents = []
for inner_list in dicts:
    documents.extend(inner_list)

In [ ]:
print(documents[:2])

In [ ]:
document_store.write_documents(documents)

In [ ]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [ ]:
from haystack.reader.farm import FARMReader
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)

In [ ]:
from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
prediction = pipe.run(query="What is the Internet Engineering Task Force?", top_k_retriever=10, top_k_reader=5)

In [ ]:
from haystack.utils import print_answers

In [ ]:
print_answers(prediction, details="minimal")